In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset, DataLoader

from PIL import Image
from glob import glob

In [3]:

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [4]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder for the input image
        self.encoder_img = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        # Encoder for the wind data
        self.encoder_wind = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        # Decoder to generate the next frame
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, img, wind):
        img_encoded = self.encoder_img(img)
        wind_encoded = self.encoder_wind(wind)
        combined = torch.cat((img_encoded, wind_encoded), dim=1)
        decoded = self.decoder(combined)
        return decoded


In [5]:
model = Autoencoder().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [21]:

class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.day_folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))])
        
        self.image_batches = []
        self.wind_data_batches = []
        

      
        for day_folder in self.day_folders:
            day_path = os.path.join(data_dir, day_folder)
            image_files = sorted([f for f in os.listdir(day_path) if f.endswith('.jpg')])
            self.image_batches.append([os.path.join(day_path, f) for f in image_files])
            
            wind_data_file = sorted([f for f in os.listdir(day_path) if f.endswith('.npy')])[0]
            wind_data_path = os.path.join(day_path, wind_data_file)
            wind_data = np.load(wind_data_path)
            
            # Ensure wind data is in the correct format (H, W, 3)
           

            if wind_data.ndim == 3 and wind_data.shape[2] == 3:
                self.wind_data_batches.append(wind_data)
            else:
                raise ValueError(f"Wind data in {day_path} is not in the correct format.")
        
    def __len__(self):
        # Total number of image pairs across all days
        return sum(len(images) - 1 for images in self.image_batches)

    def __getitem__(self, idx):
        # Determine which day folder and image pair the index refers to
        running_idx = 0
        for images, wind_data in zip(self.image_batches, self.wind_data_batches):
            num_images = len(images) - 1
            if idx < running_idx + num_images:
                img1_path = images[idx - running_idx]
                img2_path = images[idx - running_idx + 1]
                wind_data = np.repeat(wind_data[np.newaxis, :, :, :], 1, axis=0)  # Adjust to match batch size

                # Load and preprocess images
                img1 = np.array(Image.open(img1_path).resize((512, 512)).convert('RGB')) / 255.0
                img2 = np.array(Image.open(img2_path).resize((512, 512)).convert('RGB')) / 255.0
                
                # Convert to torch tensors
                img1 = torch.tensor(img1.transpose(2, 0, 1), dtype=torch.float32)
                img2 = torch.tensor(img2.transpose(2, 0, 1), dtype=torch.float32)
                wind_data = torch.tensor(wind_data.transpose(2, 0, 1), dtype=torch.float32)
                
                return img1, img2, wind_data

            running_idx += num_images

        raise IndexError("Index out of range")

# Example usage
data_dir = 'output'
train_dataset = CustomDataset(data_dir)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [23]:
from torch.utils.data import Dataset
import os
import numpy as np
from PIL import Image
import torch

class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.day_folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))])
        
        self.image_batches = []
        self.wind_data_batches = []

        for day_folder in self.day_folders:
            day_path = os.path.join(data_dir, day_folder)
            image_files = sorted([f for f in os.listdir(day_path) if f.endswith('.jpg')])
            self.image_batches.append([os.path.join(day_path, f) for f in image_files])
            
            wind_data_file = sorted([f for f in os.listdir(day_path) if f.endswith('.npy')])[0]
            wind_data_path = os.path.join(day_path, wind_data_file)
            wind_data = np.load(wind_data_path)
            
            # Ensure wind data is in the correct format (H, W, 3)
            if wind_data.ndim == 3 and wind_data.shape[2] == 3:
                self.wind_data_batches.append(wind_data)
            else:
                raise ValueError(f"Wind data in {day_path} is not in the correct format.")
        
    def __len__(self):
        # Total number of image pairs across all days
        return sum(len(images) - 1 for images in self.image_batches)

    def __getitem__(self, idx):
        # Determine which day folder and image pair the index refers to
        running_idx = 0
        for images, wind_data in zip(self.image_batches, self.wind_data_batches):
            num_images = len(images) - 1
            if idx < running_idx + num_images:
                img1_path = images[idx - running_idx]
                img2_path = images[idx - running_idx + 1]

                # Load and preprocess images
                img1 = np.array(Image.open(img1_path).resize((512, 512)).convert('RGB')) / 255.0
                img2 = np.array(Image.open(img2_path).resize((512, 512)).convert('RGB')) / 255.0
                
                # Convert to torch tensors
                img1 = torch.tensor(img1.transpose(2, 0, 1), dtype=torch.float32)
                img2 = torch.tensor(img2.transpose(2, 0, 1), dtype=torch.float32)
                
                # Select a single frame of wind data corresponding to the image pair
                wind_data_frame = wind_data[:, :, :2]  # Use x and y components
                wind_data_frame = np.expand_dims(wind_data_frame, axis=0)  # Add batch dimension

                # Convert wind data to torch tensor
                wind_data_tensor = torch.tensor(wind_data_frame.transpose(2, 0, 1), dtype=torch.float32)
                
                return img1, img2, wind_data_tensor

            running_idx += num_images

        raise IndexError("Index out of range")

# Example usage
data_dir = 'output'
train_dataset = CustomDataset(data_dir)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [42]:
pip install scipy


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------- ----------------------------- 11.8/44.5 MB 56.8 MB/s eta 0:00:01
   --------------------- ------------------ 24.1/44.5 MB 61.2 MB/s eta 0:00:01
   -------------------------------- ------- 36.7/44.5 MB 59.8 MB/s eta 0:00:01
   ---------------------------------------  44.3/44.5 MB 58.7 MB/s eta 0:00:01
   ---------------------------------------- 44.5/44.5 MB 53.5 MB/s eta 0:00:00


In [45]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
from scipy.ndimage import zoom

class CustomDataset(Dataset):
    def __init__(self, data_dir, image_size=(512, 512)):
        self.data_dir = data_dir
        self.image_size = image_size
        self.day_folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))])
        
        self.image_paths = []
        self.wind_data_batches = []

        # Load all images and wind data files
        for day_folder in self.day_folders:
            day_path = os.path.join(data_dir, day_folder)
            image_files = sorted([f for f in os.listdir(day_path) if f.endswith('.jpg')])
            self.image_paths.extend([os.path.join(day_path, f) for f in image_files])
            
            wind_data_file = sorted([f for f in os.listdir(day_path) if f.endswith('.npy')])[0]
            wind_data_path = os.path.join(day_path, wind_data_file)
            wind_data = np.load(wind_data_path)
            
            # Ensure wind data is resized to match the image size
            if wind_data.shape[:2] != self.image_size:
                # Resize using scipy
                zoom_factors = (self.image_size[0] / wind_data.shape[0], 
                                self.image_size[1] / wind_data.shape[1])
                resized_wind_data = zoom(wind_data, (zoom_factors[0], zoom_factors[1], 1), order=1)
                wind_data = resized_wind_data
            
            if wind_data.ndim == 3 and wind_data.shape[2] == 3:
                self.wind_data_batches.append(wind_data)
            else:
                raise ValueError(f"Wind data in {day_path} is not in the correct format. Expected shape (H, W, 3), got {wind_data.shape}.")
        
        # Flatten wind_data_batches to ensure proper indexing
        self.wind_data_batches = [wind_data for wind_data in self.wind_data_batches for _ in range(len(self.image_paths) // len(self.wind_data_batches))]

    def __len__(self):
        # Total number of image pairs across all days
        return len(self.image_paths) - 1

    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError("Index out of range")
        
        # Determine which image pair and corresponding wind data
        img1_path = self.image_paths[idx]
        img2_path = self.image_paths[idx + 1]
        
        # Load and preprocess images
        img1 = np.array(Image.open(img1_path).resize(self.image_size).convert('RGB')) / 255.0
        img2 = np.array(Image.open(img2_path).resize(self.image_size).convert('RGB')) / 255.0
        
        # Choose the appropriate wind data
        wind_data = self.wind_data_batches[idx % len(self.wind_data_batches)]
        
        # Convert to torch tensors
        img1 = torch.tensor(img1.transpose(2, 0, 1), dtype=torch.float32)
        img2 = torch.tensor(img2.transpose(2, 0, 1), dtype=torch.float32)
        
        try:
            wind_data_tensor = torch.tensor(wind_data.transpose(2, 0, 1), dtype=torch.float32)
        except ValueError as e:
            print(f"Error transposing wind data: {e}")
            print(f"Wind data shape before transpose: {wind_data.shape}")
            raise
        
        return img1, img2, wind_data_tensor

# Example usage
data_dir = 'output'
train_dataset = CustomDataset(data_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [47]:
# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for img1, img2, wind in train_loader:
        img1 = img1.to(device)
        img2 = img2.to(device)
        wind = wind.to(device)

        optimizer.zero_grad()
        outputs = model(img1, wind)
        loss = criterion(outputs, img2)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader)}")

# Save the trained model
model_save_path = 'D:/trained_autoencoder.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Epoch [1/20], Loss: 0.49345852539572915
Epoch [2/20], Loss: 0.4905348751746433
